In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

In [2]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['start_station','end_station','Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles',
                     'rush_hour','metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    bikeshare_machine['start_station'] = bikeshare_machine['start_station'].astype('category')
    bikeshare_machine['end_station'] = bikeshare_machine['end_station'].astype('category')
    bikeshare_machine['rush_hour'] = bikeshare_machine['rush_hour'].astype('category')
    col_names = ['start_station', 'end_station','member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles','rush_hour',
                'metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['rush_hour','member_type','holiday','work_day'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['start_station','end_station','season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [3]:
file = '/Users/matthewcassi/Documents/Bike-Sharing-Dataset/Bikeshare_Time_Prediction/Casual_RushMetro/landmarks.csv'
bs = set_data(file)

In [4]:
bs.columns

Index(['time_diff', 'temperature', 'humidity', 'windspeed', 'miles',
       'metro_dist', 'landmark_dist_start', 'landmark_dist_end', 'temp', 'hum',
       ...
       'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', 'weather_cat_1', 'weather_cat_2',
       'weather_cat_3'],
      dtype='object', length=303)

In [5]:
bs=bs.rename(columns = {'member_type_Registered':'member_type'})
bs.head()

,time_diff,temperature,humidity,windspeed,miles,metro_dist,landmark_dist_start,landmark_dist_end,temp,hum,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
894118,47.300,0.506667,0.483333,0.258083,1.350768,0.096430,0.218430,0.411781,15.813349,48.3333,...,0,0,0,0,0,0,1,1,0,0
482555,27.333,0.709167,0.742083,0.206467,0.000000,0.048442,0.060280,0.060280,25.330849,74.2083,...,1,0,0,0,0,0,0,1,0,0
42707,11.983,0.384167,0.789167,0.251871,1.078723,0.669037,0.257095,0.060280,10.055849,78.9167,...,0,0,0,0,0,0,1,0,1,0
719666,4.650,0.385833,0.595417,0.061571,0.564348,0.505960,2.314037,1.855806,10.134151,59.5417,...,0,0,1,0,0,0,0,0,1,0
673496,11.017,0.709167,0.757917,0.225129,0.996767,0.082398,0.075027,0.086349,25.330849,75.7917,...,0,0,0,0,0,1,0,0,1,0


In [6]:
X = bs.drop('member_type', 1)
y = bs['member_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((914885, 302), (304962, 302), (914885,), (304962,))

### GridSearch Logistic Regression

In [8]:
c_space = np.linspace(0.0001, 50, 20)
param_grid = {'C': c_space}

log2 = LogisticRegression()
log_gs = GridSearchCV(log2, param_grid, cv=3)
log_gs.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-04,   2.63167e+00,   5.26325e+00,   7.89482e+00,
         1.05264e+01,   1.31580e+01,   1.57895e+01,   1.84211e+01,
         2.10527e+01,   2.36843e+01,   2.63158e+01,   2.89474e+01,
         3.15790e+01,   3.42106e+01,   3.68421e+01,   3.94737e+01,
         4.21053e+01,   4.47369e+01,   4.73684e+01,   5.00000e+01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [9]:
pred2 = log_gs.predict(X_test)
score2 = log_gs.score(X_test, y_test)
train_score2 = log_gs.score(X_train, y_train)

In [10]:
score2, train_score2

(0.87726339675107068, 0.87756166075517683)

In [11]:
confusion_matrix(y_test, pred2)

array([[ 18991,  31667],
       [  5763, 248541]])

In [12]:
print(classification_report(y_test, pred2))

             precision    recall  f1-score   support

          0       0.77      0.37      0.50     50658
          1       0.89      0.98      0.93    254304

avg / total       0.87      0.88      0.86    304962



### Model 2

In [13]:
remove_cols = ['weekday_0', 'weekday_1','weekday_2','weekday_3','weekday_4','weekday_5','weekday_6', 
              'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9',
              'month_10','month_11','month_12','member_type']
X1 = np.matrix(bs.drop(remove_cols, 1))
y1 = bs['member_type']

In [14]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.25)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((914885, 283), (304962, 283), (914885,), (304962,))

In [15]:
log3 = LogisticRegression()
log_gs2 = GridSearchCV(log3, param_grid, cv=3)
log_gs2.fit(X1_train, y1_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-04,   2.63167e+00,   5.26325e+00,   7.89482e+00,
         1.05264e+01,   1.31580e+01,   1.57895e+01,   1.84211e+01,
         2.10527e+01,   2.36843e+01,   2.63158e+01,   2.89474e+01,
         3.15790e+01,   3.42106e+01,   3.68421e+01,   3.94737e+01,
         4.21053e+01,   4.47369e+01,   4.73684e+01,   5.00000e+01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [16]:
pred3 = log_gs2.predict(X1_test)
score3 = log_gs2.score(X1_test, y1_test)
train_score3 = log_gs2.score(X1_train, y1_train)

In [17]:
score3, train_score3

(0.87696171982083015, 0.87732556550823326)

In [18]:
confusion_matrix(y1_test, pred3)

array([[ 19080,  31873],
       [  5649, 248360]])

In [19]:
print(classification_report(y1_test, pred3))

             precision    recall  f1-score   support

          0       0.77      0.37      0.50     50953
          1       0.89      0.98      0.93    254009

avg / total       0.87      0.88      0.86    304962

